<a href="https://colab.research.google.com/github/drawcodeboy/first_ML_DL/blob/main/firstMLDL_05_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cross Validation & Grid Search

>* Validation Set
>* Cross Valitation
>* Hyperparameter Tuning
>* Grid Search
>* Randomized Search

In [2]:
# 데이터 가져오기

import pandas as pd
wine = pd.read_csv('http://bit.ly/wine_csv_data')

In [3]:
# 데이터 정보 가져오기

wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   sugar    6497 non-null   float64
 2   pH       6497 non-null   float64
 3   class    6497 non-null   float64
dtypes: float64(4)
memory usage: 203.2 KB


In [5]:
# input, target 분리

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [6]:
# train, test 분리

from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, random_state = 42
)

In [7]:
# train(sub), validation 분리

sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, random_state = 42
)

In [15]:
# 결정 트리 모델 선언, 훈련 세트 훈련 및 검증 세트로 검증

from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9978106185002736
0.8571428571428571


In [22]:
# Cross Validation

from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01004791, 0.00678229, 0.00733495, 0.00750184, 0.00695252]), 'score_time': array([0.00085378, 0.00067592, 0.00071907, 0.00067043, 0.00066423]), 'test_score': array([0.85128205, 0.84820513, 0.8788501 , 0.85112936, 0.84394251])}


In [24]:
# 교차 검증했을 때, 각 score의 평균 구하기

import numpy as np
print(np.mean(scores['test_score']))

0.8546818301479492


In [25]:
# Splitter(훈련 세트 섞기)를 Cross Validation

from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

0.8546818301479492


In [26]:
# Splitter 매개변수를 통해 분할 수 지정 가능 = n_splits

splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

0.8585800484734237


In [33]:
# Hyperparameter Tuning

from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [35]:
# 모델 객체 그리드 서치
# min_impurtiy_decrease 값마다 cv 기본 값이 5이므로 총 25개의 모델을 만든다.

# n_jobs는 병렬 실행에 사용할 CPU 코어 수다.
# -1로 지정하면 시스템에 있는 모든 코어를 사용한다는 뜻

gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)

In [36]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [38]:
# GridSearchCV에는 가장 점수가 높은 모델의 매개변수 조합으로 자동으로
# 모델을 다시 훈련한다.

# 이 모델은 gs 객체의 best_estimator_ 속성에 저장되어있다.

dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9137931034482759


In [40]:
# 최적의 매개변수 탐색

print(gs.best_params_)

{'min_impurity_decrease': 0.0003}


In [41]:
# 최적의 교차 검증 점수

print(gs.cv_results_['mean_test_score'])

[0.86843111 0.86925267 0.87315179 0.87212531 0.87130627]


In [42]:
# argmax는 가장 큰 값에 대한 인덱스

best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0003}


In [44]:
# cv_result_가 어떤 속성으로 이루어진 딕셔너리인지 확인
# 여러 Key를 가지고 있음
# params라는 Key 안에도 여러 Dictionary가 존재

print(gs.cv_results_)

{'mean_fit_time': array([0.00839486, 0.00856357, 0.01043258, 0.01073947, 0.00708952]), 'std_fit_time': array([0.00044858, 0.00119385, 0.00104193, 0.00274907, 0.00228627]), 'mean_score_time': array([0.00099974, 0.0009068 , 0.00097861, 0.00132275, 0.00090075]), 'std_score_time': array([8.94716537e-05, 4.04844576e-05, 1.55908659e-04, 4.88356868e-04,
       2.44085811e-04]), 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'min_impurity_decrease': 0.0001}, {'min_impurity_decrease': 0.0002}, {'min_impurity_decrease': 0.0003}, {'min_impurity_decrease': 0.0004}, {'min_impurity_decrease': 0.0005}], 'split0_test_score': array([0.87384615, 0.87076923, 0.87282051, 0.86461538, 0.86051282]), 'split1_test_score': array([0.86666667, 0.86871795, 0.87794872, 0.88512821, 0.87794872]), 'split2_test_score': array([0.88603696, 0.88295688, 0.8798768 ,

In [45]:
# 더 복잡한 매개변수 조합으로 GridSearch(Hyperparameter 튜닝)

params = {
    'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
    'max_depth': range(5, 20 ,1),
    'min_samples_split': range(2, 100, 1)
}

In [46]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100)})

In [47]:
# 최적의 매개변수 값

print(gs.best_params_)

{'max_depth': 15, 'min_impurity_decrease': 0.0001, 'min_samples_split': 22}


In [49]:
# 교차 검증 점수 중에 제일 높은 값

print(np.max(gs.cv_results_['mean_test_score']))

0.8752056020639183


In [50]:
# Random Search
# 매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있음
# 또 너무 많은 매개변수 조건이 있어 그리드 서치 수행 시간이 오래 걸림

from scipy.stats import uniform, randint

In [51]:
# randit, 정수 범위 내에서 난수를 생성하는 객체
# rvs를 통해 몇 개를 생성할 건지 지정 가능

rgen = randint(0, 10)
rgen.rvs(10)

array([4, 7, 7, 8, 1, 5, 1, 8, 9, 5])

In [52]:
# unique를 통해 확인해보면 난수들이 '균등 분포'하게 분포됨
np.unique(rgen.rvs(1000), return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 92, 106, 105, 102,  94, 104, 102,  89, 103, 103]))

In [53]:
# uniform, 실수 범위 실수 난수 생성기

ugen = uniform(0, 1)
ugen.rvs(10)

array([0.10659313, 0.49254489, 0.76841026, 0.28077236, 0.83312623,
       0.73584339, 0.91422881, 0.27083397, 0.12285257, 0.85908663])

In [54]:
# 랜덤으로 주어진 범위에서 파라미터 생성

params = {
    'min_impurity_decrease': uniform(0.0001, 0.001),
    'max_depth': randint(20, 50),
    'min_samples_split': randint(2, 25),
    'min_samples_leaf': randint(1, 25)
}

In [56]:
# Random Search 모델
# n_iter를 통해 몇 번만 Cross Validation을 하고, 최적의 파라미터를 찾을 수 있다.

from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params,
                    n_iter = 100, n_jobs = -1, random_state = 42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f45ecb30210>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f45ecb30a10>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f45ed0e76d0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f45ee2af290>},
                   random_state=42)

In [57]:
print(gs.best_params_)

{'max_depth': 42, 'min_impurity_decrease': 0.00037864646423661145, 'min_samples_leaf': 1, 'min_samples_split': 2}


In [58]:
print(np.max(gs.cv_results_['mean_test_score']))

0.874178065603117


In [59]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.8633846153846154
